# Override and ModelSettings

- Function definitions can be overridden to give more clarity.
- You can enforce tool usage to be auto, custom or none uing `ModelSettings(tool_choice=)`

In [1]:
import os

from agents import Agent, Runner, function_tool, ModelSettings
from dotenv import load_dotenv

load_dotenv()

# Disable tracing BEFORE importing agents
os.environ["OPENAI_API_KEY"] = os.getenv("OPENROUTER_API_KEY")
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"


# We can override the params in function_tool
@function_tool(
    name_override="Get_Status_of_Current_Order",
    description_override="Returns the status of the order given the order_number",
    docstring_style="Args: order_number in integer format",
)
def get_order_status(order_number: int) -> str:
    if order_number in (100, 101):
        return "Delivered"
    elif order_number in (200, 201):
        return "Delayed"
    elif order_number in (300, 301):
        return "Cancelled"
    else:
        return "Invalid order_number"


# You can enforce tool choice using
customer_service_agent = Agent(
    name="Customer Service Agent",
    instructions="You are an AI Agent that responds to customer queries for a local paper company. \
            Always use the backend system to verify do not guess.",
    model="openai/gpt-4o",
    tools=[get_order_status],
    model_settings=ModelSettings(tool_choice="required"),
)

result = await Runner.run(
    customer_service_agent, "What is the status of my order with order_number 200."
)

print(result.final_output)

The status of your order with order number 200 is "Delayed."


[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-or-v1*************************************************************13b1. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-or-v1*************************************************************13b1. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: sk-or-v1*************************************************************13b1. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "

---

# Tool Use Behaviour

- Controls what happens after a tool is called:

    1. `run_llm_again`
    2. `stop_on_first_tool` - Use when tool's o/p is the answer
    3. `agent.StopAtTools.stop_at_tool_names` - Use to immediately terminate agent execution upon a particular tool call.

In [5]:
from agents import Agent, Runner, function_tool, StopAtTools


@function_tool
def create_invoice(order_number: int) -> str:
    return (
        f"Invoice for order_number {order_number} is dated on 10th July with ID 1482829"
    )


invoice_generator_agent = Agent(
    name="Invoice Generator Agent",
    instructions="You are an AI Agent that can generate invoices for orders given the order_number",
    tools=[create_invoice],
    tool_use_behavior=StopAtTools(stop_at_tool_names=["create_invoice"]),
)

result = await Runner.run(
    invoice_generator_agent, "Generate the invoice for order_number 200"
)

print(result.final_output)

Invoice for order_number 200 is dated on 10th July with ID 1482829


---

# Pydantic

- It is used for input and output validation.

In [18]:
from pydantic import BaseModel


class RefundRequest(BaseModel):
    order_number: int
    customer_email: str
    reason: str


@function_tool
def process_refund(request: RefundRequest) -> str:
    return f"A refund of Rs 1000 has been submitted for order_number {request.order_number} with email {request.customer_email}"


refund_agent = Agent(
    name="Refund Agent",
    instructions="You are an AI Agent to process refunds for customers given order_number, customer_email and reason. \
        You should not generate random values for these params if user does not provide them.",
    model="openai/gpt-4o",
    tools=[process_refund],
    tool_use_behavior=StopAtTools(stop_at_tools=["process_refund"]),
    model_settings=ModelSettings(tool_choice="required"),
)

result = await Runner.run(
    refund_agent,
    "Please provess refund for order_number-100, customer_email-idx@gmail.com and reason-poor quality",
)

print(result.final_output)

The refund of Rs 1000 for the order associated with the email address idx@gmail.com has been successfully processed due to the reason of poor quality.
